In [1]:
import torch
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer
from tqdm import tqdm
import json
import gc
import os
from datasets import load_from_disk

from config import storage_dir

# Setting arguments and paths

In [2]:
model_name = "Llama-3.3-70B-Instruct" #Qwen/Qwen3-14B"
model_storage_dir = os.path.join(storage_dir, "lm_sys", model_name.split("/")[-1])
response_path = os.path.join(model_storage_dir, 'lm_sys_responses')

# Loading and examining dataset

In [3]:
dataset = load_from_disk(response_path)
print(dataset)

Dataset({
    features: ['conversation'],
    num_rows: 128
})


In [5]:
dataset[1]

{'conversation': [{'content': "Beside OFAC's selective sanction that target the listed individiuals and entities, please elaborate on the other types of US's sanctions, for example, comprehensive and sectoral sanctions. Please be detailed as much as possible",
   'role': 'user'},
  {'content': "The United States employs a range of sanctions tools to achieve its foreign policy and national security objectives. Besides the selective or targeted sanctions imposed by the Office of Foreign Assets Control (OFAC), which focus on specific individuals and entities, the U.S. government also implements comprehensive and sectoral sanctions. Each type of sanction has a distinct approach and impact on the targeted country, entity, or individual.\n\n### 1. Comprehensive Sanctions\n\nComprehensive sanctions, also known as country-based sanctions, are the most severe form of economic sanctions. They are imposed on an entire country, effectively isolating it from the global economy. These sanctions typi

In [4]:
with open(os.path.join(response_path, 'generation_params.json'), 'r') as f:
    generation_params = json.load(f)
print(generation_params)

{'context_length': 512, 'start_idx': 0, 'max_samples': 10500, 'model_name': 'Qwen/Qwen3-14B', 'batch_size': 128, 'gpu_count': 4, 'processing_time': 34.9708014011383}


In [5]:
def count_tokens(text, tokenizer):
    return len(tokenizer.encode(text, add_special_tokens=False))

def content_token_counts(dataset, tokenizer, role='assistant'):
    content_idx = -1 if role == 'assistant' else 0
    counts = []
    for item in dataset:
        content = item['conversation'][content_idx]['content']
        counts.append(count_tokens(content, tokenizer))
    return counts

tokenizer = AutoTokenizer.from_pretrained(model_name)
counts = content_token_counts(dataset, tokenizer)

In [6]:
import numpy as np
print(np.sum(np.array(counts) ==512)/len(counts))  # Didn't reach EOS

0.33076190476190476


In [7]:
dataset[0]

{'conversation': [{'content': 'how can identity protection services help protect me against identity theft',
   'role': 'user'},
  {'content': "Identity protection services are designed to help you safeguard your personal information and reduce the risk of identity theft. These services can be especially valuable in today's digital age, where personal data is frequently collected, stored, and sometimes misused. Here's how identity protection services can help protect you against identity theft:\n\n---\n\n### 1. **Monitoring for Suspicious Activity**\nIdentity protection services continuously monitor your personal information across various databases, credit reports, and online platforms. This includes:\n\n- **Credit monitoring**: Alerts you to new credit accounts opened in your name.\n- **Public record monitoring**: Tracks changes in your personal information (e.g., address, phone number, marriage, divorce, etc.).\n- **Dark web monitoring**: Scans the dark web for your personal informa

# Conversion to ROT13

In [42]:
import re
import codecs

def rot13_alpha(text):
    # Apply ROT13 only to alphabetic substrings
    def rot13_match(match):
        return codecs.encode(match.group(0), 'rot_13')
    # Substitute only [A-Za-z]+ substrings
    return re.sub(r'[A-Za-z]+', rot13_match, text)

def rot13_dataset(dataset):
    new_dataset = []
    n_items = 0
    for item in dataset:
        new_item = item.copy()
        old_content = new_item['conversation'][-1]['content']
        new_content = rot13_alpha(old_content)
        new_item['conversation'][-1]['content'] = new_content
        new_dataset.append(new_item)
        n_items += 1
    return new_dataset

In [43]:
new_dataset = rot13_dataset(dataset)
new_hf_dataset = Dataset.from_list(new_dataset)
rot13_save_path = os.path.join(model_storage_dir, 'lm_sys_responses_rot13')
new_hf_dataset.save_to_disk(rot13_save_path)

Saving the dataset (0/1 shards):   0%|          | 0/10500 [00:00<?, ? examples/s]

In [50]:
counts = content_token_counts(new_dataset, tokenizer)
counts = np.array(counts)
counts.max()

np.int64(1535)

In [51]:
counts = content_token_counts(new_dataset, tokenizer, role='user')
counts = np.array(counts)
counts.max()

np.int64(2560)

In [52]:
2560+1535

4095

In [54]:
np.median(counts)

np.float64(24.0)